In [ ]:
# finished importing data, no depth integration started on dataset 2/
# dataset 1/ and 3/ not yet included

In [ ]:
# Go Jan 2022
#
# Purpose: combine chl data from measurements in SoG
#
# Data in:
#     1/ DFO / IOS Chl data downloaded from waterproperties.ca in 2020,
#            1979 - 2000
#            .CSV
#             xyzt
#            filtered for any records with chl or flourescence
#            converted by Dhavan at PSF in 2020 to CSV
#     2/ DFO / IOS sample data acquired by E Olson EOAS
#            1980 - 2014
#            n = 
#            sqllite db
#            seems to have exluded some Chl_a measurements as compared to 1/
#            (may have queried for nutrient data and therefore excluded some)
#
#     3/ depth integrated Chl_a from Rich Pawlowicz and Cit Sci program
#            2015-2020
#            See Pawlowicz et al (2020) SoG Atlas for methods
#            (depth integration involved a multiplier downward on Chl-a of 0.6)
#
# Data out: 
#     1/ unified dataset


In [2]:
import pandas as pd
import numpy as np
import datetime as dt 
import geopandas as gpd
#import geoplot
import matplotlib.pyplot as plt 
from netCDF4 import Dataset
import os

#import sys
#!conda install --yes --prefix {sys.prefix} geopandas

#from salishsea_tools import geo_tools as gt
#from salishsea_tools import tidetools as tt
from salishsea_tools import evaltools as et
# to install and use salishsea-tools you may need to open Jupyter from Anaconda, specifically, 
# instead of standalone notebook from 'start menu'.
# To install salishsea tools clone the git locally and use anaconda powershell 
# go to SalishSeaCast/tools folder

# conda env create -f SalishSeaTools/environment.yaml
# activate salishsea-tools
# pip install --editable SalishSeaTools

# https://salishsea-meopar-tools.readthedocs.io/en/latest/SalishSeaTools/development.html#getting-the-code
# https://salishsea-meopar-tools.readthedocs.io/en/latest/SalishSeaTools/api.html#module-salishsea_tools.geo_tools
# salishsea_tools.geo_tools.closestPointArray(lons, lats, model_lons, model_lats, tol2=1, grid='NEMO', land_mask=None, tols={'GEM2.5': {'tol_lat': 0.012, 'tol_lon': 0.016}, 'NEMO': {'tol_lat': 0.00388, 'tol_lon': 0.0104}})
# salishsea_tools.geo_tools.distance_along_curve(lons, lats)
# salishsea_tools.geo_tools.find_closest_model_point(lon, lat, model_lons, model_lats, grid='NEMO', land_mask=None, tols={'GEM2.5': {'tol_lat': 0.012, 'tol_lon': 0.016}, 'NEMO': {'tol_lat': 0.00388, 'tol_lon': 0.0104}}, checkTol=False)
# salishsea_tools.geo_tools.haversine(lon1, lat1, lon2, lat2)



In [94]:
# IOS observational data from DFO, parsed to SQLLite by E Olson (Oct, 2020)
path = "C://Users//Greig//Sync//6. SSMSP Model//Model Greig//Data//28. Phytoplankton//WaterPropsSoG_DFO-Olson_1980-2014//ORIGINAL//"
dbname='DFO_OcProfDB.sqlite'

# replaces need for sqlalchemy (see below) using Salish Sea Tools
#https://github.com/SalishSeaCast/tools/blob/master/SalishSeaTools/salishsea_tools/evaltools.py

# load DFO data automatically
yearmin = 1979
monthmin = 2
daymin = 1
yearmax = 2018
monthmax=12
daymax=30

# filter dfo - depth, year
maxdepth = 40
maxyear = 2018

datelims=(dt.datetime(yearmin,monthmin,daymin),
          dt.datetime(yearmax,monthmax,daymax))
excludeSaanich=True

try:
    from sqlalchemy import create_engine, case, MetaData, event
    from sqlalchemy.orm import *
    from sqlalchemy.ext.automap import automap_base
    from sqlalchemy.sql import and_, or_, not_, func
    from sqlalchemy.ext.declarative import declarative_base, DeferredReflection
except ImportError:
    raise ImportError('You need to install sqlalchemy in your environment to use this function.')

if not os.path.isfile(os.path.join(path, dbname)):
    raise Exception('ERROR: {}.sqlite does not exist'.format(dbname))
engine = sqlalchemy.create_engine('sqlite:///' + path + dbname, echo = True)


# get metadata so you can see some information about the tables loaded above
md=MetaData()
md.reflect(engine)

# list the fields present in the relevant tables:
print('Columns in StationTBL:')
print(sorted([x.name for x in md.tables['StationTBL'].columns]))
print('Columns in ObsTBL:')
print(sorted([x.name for x in md.tables['ObsTBL'].columns]))
# note: the fields in CalcsTBL were calculated from values in ObsTBL and were not present in the original data
print('Columns in CalcsTBL:')
print(sorted([x.name for x in md.tables['CalcsTBL'].columns]))


# following code from evaltools, create pandas df object using sqlalchemy
# automap creates an object from an sql database -GO
#https://docs.sqlalchemy.org/en/14/orm/extensions/automap.html
Base = automap_base()
# reflect the tables 
Base.prepare(engine, reflect=True)
# mapped classes have been created
# existing tables:
StationTBL=Base.classes.StationTBL
ObsTBL=Base.classes.ObsTBL
CalcsTBL=Base.classes.CalcsTBL
session = create_session(bind = engine, autocommit = False, autoflush = True)
SA=case([(CalcsTBL.Salinity_Bottle_SA!=None, CalcsTBL.Salinity_Bottle_SA)], else_=
        case([(CalcsTBL.Salinity_T0_C0_SA!=None, CalcsTBL.Salinity_T0_C0_SA)], else_=
             case([(CalcsTBL.Salinity_T1_C1_SA!=None, CalcsTBL.Salinity_T1_C1_SA)], else_=
                  case([(CalcsTBL.Salinity_SA!=None, CalcsTBL.Salinity_SA)], else_=
                       case([(CalcsTBL.Salinity__Unknown_SA!=None, CalcsTBL.Salinity__Unknown_SA)],
                            else_=CalcsTBL.Salinity__Pre1978_SA)
                      ))))
Tem=case([(ObsTBL.Temperature!=None, ObsTBL.Temperature)], else_=
         case([(ObsTBL.Temperature_Primary!=None, ObsTBL.Temperature_Primary)], else_=
              case([(ObsTBL.Temperature_Secondary!=None, ObsTBL.Temperature_Secondary)], else_=ObsTBL.Temperature_Reversing)))
TemUnits=case([(ObsTBL.Temperature!=None, ObsTBL.Temperature_units)], else_= 
              case([(ObsTBL.Temperature_Primary!=None, ObsTBL.Temperature_Primary_units)], else_=
                   case([(ObsTBL.Temperature_Secondary!=None, ObsTBL.Temperature_Secondary_units)],
                        else_=ObsTBL.Temperature_Reversing_units)))
TemFlag=ObsTBL.Quality_Flag_Temp
CT=case([(CalcsTBL.Temperature_CT!=None, CalcsTBL.Temperature_CT)], else_=
        case([(CalcsTBL.Temperature_Primary_CT!=None, CalcsTBL.Temperature_Primary_CT)], else_=
             case([(CalcsTBL.Temperature_Secondary_CT!=None, CalcsTBL.Temperature_Secondary_CT)],
                  else_=CalcsTBL.Temperature_Reversing_CT)
            ))

if len(datelims)<2:
    qry=session.query(StationTBL.StartYear.label('Year'),StationTBL.StartMonth.label('Month'),
                      StationTBL.StartDay.label('Day'),StationTBL.StartHour.label('Hour'),
                      StationTBL.Lat,StationTBL.Lon,
                      ObsTBL.Pressure,ObsTBL.Depth,ObsTBL.Chlorophyll_Extracted,
                      ObsTBL.Chlorophyll_Extracted_units,ObsTBL.Nitrate_plus_Nitrite.label('N'),
                      ObsTBL.Silicate.label('Si'),ObsTBL.Silicate_units,SA.label('AbsSal'),CT.label('ConsT'),
                      ObsTBL.Oxygen_Dissolved,ObsTBL.Oxygen_Dissolved_units).\
    select_from(StationTBL).join(ObsTBL,ObsTBL.StationTBLID==StationTBL.ID).\
    join(CalcsTBL,CalcsTBL.ObsID==ObsTBL.ID).filter(and_(StationTBL.Lat>47-3/2.5*(StationTBL.Lon+123.5),
                                                         StationTBL.Lat<47-3/2.5*(StationTBL.Lon+121)))
else:
    start_y=datelims[0].year
    start_m=datelims[0].month
    start_d=datelims[0].day
    end_y=datelims[1].year
    end_m=datelims[1].month
    end_d=datelims[1].day
    # unresolved - column EVENT NUMBER and WATER DEPTH have spaces nd can't be queried
    qry=session.query(StationTBL.AGENCY,  StationTBL.ID, StationTBL.PROJECT, 
                      StationTBL.StartYear.label('Year'),StationTBL.StartMonth.label('Month'),
                      StationTBL.StartDay.label('Day'),StationTBL.StartHour.label('Hour'),
                      StationTBL.Lat,StationTBL.Lon,
                      ObsTBL.Pressure,ObsTBL.Depth,ObsTBL.Chlorophyll_Extracted,
                      ObsTBL.Chlorophyll_Extracted_units,ObsTBL.Nitrate_plus_Nitrite.label('N'),
                      ObsTBL.Silicate.label('Si'),ObsTBL.Silicate_units,SA.label('AbsSal'),CT.label('ConsT'),
                      ObsTBL.Oxygen_Dissolved,ObsTBL.Oxygen_Dissolved_units).\
    select_from(StationTBL).join(ObsTBL,ObsTBL.StationTBLID==StationTBL.ID).\
    join(CalcsTBL,CalcsTBL.ObsID==ObsTBL.ID).filter(and_(or_(StationTBL.StartYear>start_y,
                                                             and_(StationTBL.StartYear==start_y, StationTBL.StartMonth>start_m),
                                                             and_(StationTBL.StartYear==start_y, StationTBL.StartMonth==start_m, StationTBL.StartDay>=start_d)),
                                                         or_(StationTBL.StartYear<end_y,
                                                             and_(StationTBL.StartYear==end_y,StationTBL.StartMonth<end_m),
                                                             and_(StationTBL.StartYear==end_y,StationTBL.StartMonth==end_m, StationTBL.StartDay<end_d)),
                                                         StationTBL.Lat>47-3/2.5*(StationTBL.Lon+123.5),
                                                         StationTBL.Lat<47-3/2.5*(StationTBL.Lon+121)))#,
                                                                    #not_(and_(StationTBL.Lat>48.77,StationTBL.Lat<49.27,
                                                                    #          StationTBL.Lon<-123.43))))
if excludeSaanich:
    qry1=qry.filter(not_(and_(StationTBL.Lat>48.47,StationTBL.Lat<48.67, 
                              StationTBL.Lon>-123.6,StationTBL.Lon<-123.43)))
    df1=pd.read_sql_query(qry1.statement, engine)
else:
    df1=pd.read_sql_query(qry.statement, engine)
    df1['Z']=np.where(df1['Depth']>=0,df1['Depth'],-1.0*gsw.z_from_p(p=df1['Pressure'].values,lat=df1['Lat'].values))
    df1['dtUTC']=[dt.datetime(int(y),int(m),int(d))+dt.timedelta(hours=h) for ind, (y,m,d,h) in df1.loc[:,['Year','Month','Day','Hour']].iterrows()]
    session.close()
    engine.dispose()

df1

#print(type(df1))
#print(df1.dtypes)
#print("table length:")
#print(len(df1))
#print("Depth is null count:")
#print(df1['Depth'].isnull().sum())
#print("Pressure is null count:")
#print(df1['Pressure'].isnull().sum())


# Most records don't have explicit depth, use pressure instead
print("Missing depths before:")
print(df1['Depth'].isnull().sum())
df1.loc[df1.Depth.isnull(), 'Depth'] = df1.Pressure
print("After:")
print(df1['Depth'].isnull().sum())

#obs_df = pd.DataFrame(obs_series)
#print(df1.tail(n=20))

print(df1_chl.AGENCY.unique())

# drop nulls from depth and chl
df1_chl = df1.dropna(axis=0, how="any", thresh=None, subset=['Chlorophyll_Extracted','Depth'], inplace=False)
df1_chl

2022-01-27 14:45:16,169 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-01-27 14:45:16,169 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-27 14:45:16,173 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("CalcsTBL")
2022-01-27 14:45:16,177 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-27 14:45:16,185 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-01-27 14:45:16,185 INFO sqlalchemy.engine.Engine [raw sql] ('CalcsTBL',)
2022-01-27 14:45:16,189 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("CalcsTBL")
2022-01-27 14:45:16,189 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-27 14:45:16,193 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-01-27 14:45:16,193 INFO sqlalchemy.engine.Engine [raw

2022-01-27 14:45:16,477 INFO sqlalchemy.engine.Engine [raw sql] ('JDFLocsTBL',)
2022-01-27 14:45:16,481 INFO sqlalchemy.engine.Engine PRAGMA main.index_list("JDFLocsTBL")
2022-01-27 14:45:16,481 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-27 14:45:16,485 INFO sqlalchemy.engine.Engine PRAGMA temp.index_list("JDFLocsTBL")
2022-01-27 14:45:16,485 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-27 14:45:16,485 INFO sqlalchemy.engine.Engine PRAGMA main.index_list("JDFLocsTBL")
2022-01-27 14:45:16,489 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-27 14:45:16,489 INFO sqlalchemy.engine.Engine PRAGMA temp.index_list("JDFLocsTBL")
2022-01-27 14:45:16,489 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-27 14:45:16,493 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-01-27 14:45:16,493 INFO sqlalchemy.engine.Engine [raw sql] ('JDFLocsTBL',)
2022-01-27 14:45:16,497 INFO

,AGENCY,ID,PROJECT,Year,Month,Day,Hour,Lat,Lon,Pressure,Depth,Chlorophyll_Extracted,Chlorophyll_Extracted_units,N,Si,Silicate_units,AbsSal,ConsT,Oxygen_Dissolved,Oxygen_Dissolved_units
0,"IOS, Ocean Ecology, Sidney, B.C.",1,Strait of Georgia Pr,1982.0,7.0,16.0,1.350000,49.250000,-123.943000,NaN,12.0,5.28,mg/m^3,6.60,13.60,umol/L,NaN,NaN,NaN,None
1,"IOS, Ocean Ecology, Sidney, B.C.",1,Strait of Georgia Pr,1982.0,7.0,16.0,1.350000,49.250000,-123.943000,NaN,21.5,0.61,mg/m^3,21.20,45.00,umol/L,NaN,NaN,NaN,None
9,"IOS, Ocean Sciences Division, Sidney, B.C.",37,Line P,2007.0,8.0,29.0,20.016111,51.678000,-127.333333,5.1,5.1,7.03,mg/m^3,17.00,23.00,umol/L,29.046929,9.528417,NaN,None
10,"IOS, Ocean Sciences Division, Sidney, B.C.",38,Line P,2007.0,8.0,29.0,21.636111,51.675667,-127.293000,5.5,5.5,6.60,mg/m^3,17.90,23.80,umol/L,29.041717,9.624807,NaN,None
11,"IOS, Ocean Sciences Division, Sidney, B.C.",46,SoG/JdF,2008.0,6.0,20.0,11.473056,48.470000,-124.547167,2.0,2.0,5.30,mg/m^3,21.30,37.70,umol/L,31.731979,9.648685,6.158,mL/L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29157,"IOS, Ocean Sciences Division, Sidney, B.C.",9085,SoG / Van Harbour Water Properties Survey,2018.0,2.0,6.0,17.620000,49.299333,-122.983000,10.1,10.0,0.25,mg/m^3,25.47,57.20,umol/L,25.807440,8.116158,238.200,umol/kg
29158,"IOS, Ocean Sciences Division, Sidney, B.C.",9085,SoG / Van Harbour Water Properties Survey,2018.0,2.0,6.0,17.620000,49.299333,-122.983000,20.3,20.1,0.24,mg/m^3,25.61,57.54,umol/L,26.294613,8.169118,235.500,umol/kg
29163,"IOS, Ocean Sciences Division, Sidney, B.C.",9091,SoG / Van Harbour Water Properties Survey,2018.0,2.0,7.0,1.859167,49.304833,-122.941167,1.4,1.4,0.20,mg/m^3,15.92,48.65,umol/L,18.158550,7.712114,295.000,umol/kg
29166,"IOS, Ocean Sciences Division, Sidney, B.C.",9091,SoG / Van Harbour Water Properties Survey,2018.0,2.0,7.0,1.859167,49.304833,-122.941167,9.9,9.8,0.20,mg/m^3,25.70,59.31,umol/L,25.947112,8.260733,214.100,umol/kg


In [ ]:
# fix 

In [89]:
# read other IOS data
StationTBL.columns()

AttributeError: type object 'StationTBL' has no attribute 'columns'

In [78]:
df1_chl.AGENCY.unique()

array(['IOS, Ocean Ecology, Sidney, B.C.',
       'IOS, Ocean Sciences Division, Sidney, B.C.',
       'PBS, Ocean Science and Productivity, Sidney, B.C.',
       'IOS, Ocean Science and Productivity, Sidney, BC',
       'IOS, Ocean Science and Productivity. Sidney, BC',
       'IOS, Ocean Science and Productivity, Sidney, B.C.',
       'IOS, Ocean Sciences Division, Sidney',
       'IOS, Ocean Sciences Division, Sidney, BC',
       'PBS, Ecosystem Sciences Division, Nanaimo, B.C.'], dtype=object)

In [72]:
# problem is that depths are irregular and to integrate across depths assumptions must be made: 
# (1) the maximum depth where there would be chlorophyll
# (2) the decay or interpolation function to fill in missing chl values
# there are also calibration issues (see Pawlowicz et al 2020 because
# Non-photochemical quenching results in a light-dependent reduction of the fluorescence signal, 
# relative to measurements made in darkness - overestimateion of chl-a in daytime requiring adjustment. 
# Pawlowicz adjust using a conversion factor of 0.6)
# (3) the non-photochemical quenching must be accounted for

df1_chl 



,Year,Month,Day,Hour,Lat,Lon,Pressure,Depth,Chlorophyll_Extracted,Chlorophyll_Extracted_units,N,Si,Silicate_units,AbsSal,ConsT,Oxygen_Dissolved,Oxygen_Dissolved_units
0,1982.0,7.0,16.0,1.350000,49.250000,-123.943000,NaN,12.0,5.28,mg/m^3,6.60,13.60,umol/L,NaN,NaN,NaN,None
1,1982.0,7.0,16.0,1.350000,49.250000,-123.943000,NaN,21.5,0.61,mg/m^3,21.20,45.00,umol/L,NaN,NaN,NaN,None
9,2007.0,8.0,29.0,20.016111,51.678000,-127.333333,5.1,5.1,7.03,mg/m^3,17.00,23.00,umol/L,29.046929,9.528417,NaN,None
10,2007.0,8.0,29.0,21.636111,51.675667,-127.293000,5.5,5.5,6.60,mg/m^3,17.90,23.80,umol/L,29.041717,9.624807,NaN,None
11,2008.0,6.0,20.0,11.473056,48.470000,-124.547167,2.0,2.0,5.30,mg/m^3,21.30,37.70,umol/L,31.731979,9.648685,6.158,mL/L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29157,2018.0,2.0,6.0,17.620000,49.299333,-122.983000,10.1,10.0,0.25,mg/m^3,25.47,57.20,umol/L,25.807440,8.116158,238.200,umol/kg
29158,2018.0,2.0,6.0,17.620000,49.299333,-122.983000,20.3,20.1,0.24,mg/m^3,25.61,57.54,umol/L,26.294613,8.169118,235.500,umol/kg
29163,2018.0,2.0,7.0,1.859167,49.304833,-122.941167,1.4,1.4,0.20,mg/m^3,15.92,48.65,umol/L,18.158550,7.712114,295.000,umol/kg
29166,2018.0,2.0,7.0,1.859167,49.304833,-122.941167,9.9,9.8,0.20,mg/m^3,25.70,59.31,umol/L,25.947112,8.260733,214.100,umol/kg


In [53]:
df1_chl = df1['Chlorophyll_Extracted'].dropnna


24515